# Import thư viện

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

# Đọc dữ liệu

In [ ]:
df = pd.read_csv('/content/book_data_cleaned.csv')
df.head()

,product_id,title,authors,original_price,current_price,quantity,category,n_review,avg_rating,pages,manufacturer
0,74021317,Cây Cam Ngọt Của Tôi,José Mauro de Vasconcelos,108000,64800,53075,Tiểu Thuyết,11481,5.0,244,Nhà Xuất Bản Hội Nhà Văn
1,184466860,Hành Tinh Của Một Kẻ Nghĩ Nhiều,Nguyễn Đoàn Minh Thư,86000,59900,7929,Sách tư duy - Kỹ năng sống,780,4.8,184,Nhà Xuất Bản Thế Giới
2,73787185,Những Tù Nhân Của Địa Lý,Tim Marshall,210000,126000,17896,Lĩnh vực khác,3623,4.8,430,Nhà Xuất Bản Hội Nhà Văn
3,52789367,Nhà Giả Kim (Tái Bản 2020),Paulo Coelho,79000,47400,24668,Tác phẩm kinh điển,5131,5.0,228,Nhà Xuất Bản Hà Nội
4,147920903,Một Thoáng Ta Rực Rỡ Ở Nhân Gian,Không rõ,135000,81000,10000,Tiểu Thuyết,1636,4.8,304,Nhà Xuất Bản Hội Nhà Văn


# Tiền xử lý dữ liệu

In [ ]:
# Loại bỏ ngoại lệ
q1 = df['quantity'].quantile(0.25)
q3 = df['quantity'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
df = df[(df['quantity'] >= lower_bound) & (df['quantity'] <= upper_bound)]

# Áp dụng Label Encoding
label_encoder = LabelEncoder()
df['category'] = label_encoder.fit_transform(df['category'])

# Chọn các thuộc tính
# Qua quá trình hiểu dữ liệu, nhóm chọn 3 thuộc tính có ảnh hưởng đến số lượng bán ra là current_price, category và n_review
X = df[['current_price', 'category', 'n_review']]
y = df['quantity']

<ipython-input-52-c6200f79d23f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category'] = label_encoder.fit_transform(df['category'])


In [ ]:
# Chuẩn hóa đữ liệu
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# Chia dữ liệu thành các tập train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Huấn luyện mô hình

Vì dữ liệu có mối quan hệ tuyến tính không mạnh nên nhóm quyết định sử dụng các mô hình giải quyết được các mối quan hệ phức tạp. XGBoost là một trong những mô hình hiệu quả nhất.

In [ ]:
model = XGBRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

# Đánh giá mô hình

In [ ]:
# Dự đoán tập test
y_pred = model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)
mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)
r2 = r2_score(y_test, y_pred)
print("R² Score:", r2)

MSE: 323864.7972073924
MAE: 318.82455114650116
R² Score: 0.7655742168426514


In [ ]:
print("Trung bình số lượng bán ra:", y.mean())
print("Phương sai số lượng bán ra:", y.var())

Trung bình số lượng bán ra: 1022.7994891443167
Phương sai số lượng bán ra: 1557857.0882042092


# Cải tiến

In [ ]:
# Nhóm quyết định loại bỏ thuộc tính current_price
X = df[['category', 'n_review']]
y = df['quantity']

# Chuẩn hóa đữ liệu
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Chia dữ liệu thành các tập train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = XGBRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

# Đánh giá mô hình

In [ ]:
# Dự đoán tập test
y_pred = model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)
mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)
r2 = r2_score(y_test, y_pred)
print("R² Score:", r2)

MSE: 293609.4253309373
MAE: 295.9232428836974
R² Score: 0.7874742746353149


In [ ]:
print("Trung bình số lượng bán ra:", y.mean())
print("Phương sai số lượng bán ra:", y.var())

Trung bình số lượng bán ra: 1022.7994891443167
Phương sai số lượng bán ra: 1557857.0882042092


# Phân tích kết quả

- Khi giữ lại 2 thuộc tính (category, n_review), MSE giảm từ 323,864.80 xuống 293,609.43. Điều này cho thấy việc loại bỏ current_price giúp mô hình tập trung tốt hơn vào hai thuộc tính còn lại.
- MAE giảm từ 318.82 xuống 295.92 cho thấy sai số trung bình giảm khoảng 22.9 đơn vị. Mức giảm này đáng kể cho thấy mô hình khi loại bỏ thuộc tính current_price đã đưa ra dự đoán gần đúng hơn.
- R2 tăng từ 0.7656 lên 0.7875 phản ánh rằng mô hình với 2 thuộc tính (category, n_review) giải thích được 78.75% sự biến thiên của dữ liệu, cao hơn so với mô hình sử dụng 3 thuộc tính.
- Kết luận: Loại bỏ thuộc tính current_price cải thiện hiệu suất mô hình. Thuộc tính current_price có thể chứa thông tin nhiễu hoặc không có mối liên hệ chặt chẽ với mục tiêu quantity.
